<a href="https://colab.research.google.com/github/tanuja200/PGCPtest/blob/main/Copy_of_H1_Speech_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://cdn.iiith.talentsprint.com/aiml/Hackathon_data/studio_rev_data_01.zip
!unzip -qq studio_rev_data_01.zip 

--2021-07-30 05:47:04--  https://cdn.iiith.talentsprint.com/aiml/Hackathon_data/studio_rev_data_01.zip
Resolving cdn.iiith.talentsprint.com (cdn.iiith.talentsprint.com)... 172.105.52.210
Connecting to cdn.iiith.talentsprint.com (cdn.iiith.talentsprint.com)|172.105.52.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61767991 (59M) [application/zip]
Saving to: ‘studio_rev_data_01.zip’

studio_rev_data_01. 100%[===================>]  58.91M  7.65MB/s    in 9.8s    

2021-07-30 05:47:15 (5.98 MB/s) - ‘studio_rev_data_01.zip’ saved [61767991/61767991]



In [ ]:
import scipy.io as sio
import os
# Importing torch packages
import torch
import torch.nn as nn      
import torch.nn.functional as F
import torch.optim as optim

# Importing python packages
import numpy as np

# Import glob to perform pattern matching to find files
import glob
from torch.autograd import Variable

import librosa

Feature extraction method 3

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
       
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

audio_labels = [0, 1, 2, 3, 4, 5]

def load_data_1():
    
    x,y, z=[],[], []
  
    for file in glob.glob("/content/studio_data/*.wav", recursive=True):
        
      file_name=os.path.basename(file)
      
        #emotion = emotions[file_name.split("-")[2]]
      labels = (int((file_name.split("/")[-1]).split("_")[0]))
      if labels in audio_labels:
          feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
          x.append(feature)
          y.append(labels)
          z.append(file_name)
    return x, y, z

import soundfile
x, y, z= load_data_1()

/usr/local/lib/python3.7/dist-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


SKLEARN Classifier


In [ ]:
features = np.array(x)
labels = np.array(y)

In [ ]:
features.shape

(3979, 180)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
labels = lb.fit_transform(labels)
lb.classes_


array([0, 1, 2, 3, 4, 5])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [ ]:
# NORMALIZE DATA
#mean = np.mean(features, axis=0)
#std = np.std(features, axis=0)
#features = (features - mean)/std

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state=42, shuffle=False)

In [ ]:
x2={}
for i in list(y_train):
  x2[i] = list(y_train).count(i)

x2

{0: 508, 1: 533, 2: 532, 3: 534, 4: 511, 5: 565}

In [ ]:
y2={}
for i in list(y_test):
  y2[i] = list(y_test).count(i)

y2

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Create a object for MLPClassifier 
clf = MLPClassifier()

# Fit the train fetaures and labels to the classifier object
clf.fit(np.array(X_train), np.array(y_train))  

# Predict the labels of the test features
predicted_values = clf.predict(X_test) 

In [ ]:
# Get the accuracy score by comparing predicted labels and test labels
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predicted_values)

# Pytorch Convolutional layers

In [ ]:
import torch
from torchvision import datasets, transforms, models
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

In [ ]:
# RESHAPE DATA TO INCLUDE 3D TENSOR 
X_train_1 = X_train[:,:,np.newaxis]
X_test_2 = X_test[:,:,np.newaxis]

X_train.shape, X_test.shape, X_train_1.shape, X_test_2.shape

((3183, 180), (796, 180), (3183, 180, 1), (796, 180, 1))

In [ ]:
batch_size = 200
# The TensorDataset takes an arbitrary number of input tensors.
train_dataset_1 = torch.utils.data.TensorDataset(torch.FloatTensor(X_train_1),torch.LongTensor(y_train))
test_dataset_1 = torch.utils.data.TensorDataset(torch.FloatTensor(X_test_2),torch.LongTensor(y_test))

# Loading the train dataset
train_loader_1 = torch.utils.data.DataLoader(train_dataset_1, batch_size=batch_size, num_workers=2)
test_loader_1 = torch.utils.data.DataLoader(test_dataset_1, batch_size=batch_size, num_workers=2) 

In [ ]:
for batch in iter(train_loader_1):
  print(batch[0].size())

torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([200, 180, 1])
torch.Size([183, 180, 1])


In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=180, out_channels=120, kernel_size=1 )
        self.bn1 = nn.BatchNorm1d(120)
        self.pool1 = nn.MaxPool1d(1)
        self.conv2 = nn.Conv1d(120, 100, 1)
        self.bn2 = nn.BatchNorm1d(100)
        self.pool2 = nn.MaxPool1d(1)
        self.conv3 = nn.Conv1d(100, 128, 1)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(1)
        #self.conv4 = nn.Conv1d(128, 128, 1)
        #self.bn4 = nn.BatchNorm1d(128)
        #self.pool4 = nn.MaxPool1d(1)
        #self.conv5 = nn.Conv1d(256, 128, 1)
        #self.bn5 = nn.BatchNorm1d(128)
        #self.pool5 = nn.MaxPool1d(1)
        self.fc1 = nn.Linear(128, 32)
        #self.fc2 = nn.Linear(128, 64)
        #self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 6)
        self.dropout = nn.Dropout(p=0.20)

    def forward(self, x):
   
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool1(x)
        
        x = self.dropout(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool2(x)
       
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool3(x)
        
        #x = self.conv4(x)
        #x = self.bn4(x)
        #x = F.relu(x)
        #x = self.pool4(x)
        #x = self.dropout(x)

        #x = self.conv5(x)
        #x = self.bn5(x)
        #x = self.pool5(x)
        
        #x = self.dropout(x)
       
        x = x.view(-1, 128)
        x = self.fc1(x)
        #x = self.fc2(x)
        #x = self.fc3(x)
        x = self.fc4(x)
        #x = F.log_softmax(x, dim=1)
        return x      
      

In [ ]:
print(torch.cuda.is_available())

device  =  torch.device("cuda" if torch.cuda.is_available() else "cpu")

False


In [ ]:
cnn_model = CNNModel()
#model = M5()
print(cnn_model)
cnn_model= cnn_model.to(device)
# YOUR CODE HERE to convert your model to runtime cuda

criterion = nn.CrossEntropyLoss()# YOUR CODE HERE : Explore and declare loss function

optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)

CNNModel(
  (conv1): Conv1d(180, 120, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(120, 100, kernel_size=(1,), stride=(1,))
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(100, 128, kernel_size=(1,), stride=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=6, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [ ]:
num_epochs = 20
cnn_model.train()

#Define the lists to store the results of loss and accuracy
train_loss = []
train_accuracy, test_accuracy = [], []

for epoch in range(num_epochs): 
    correct = 0
    iter_loss = 0.0
    
    for (inputs, labels) in train_loader_1:

        inputs = inputs.to(device)
        train_labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = cnn_model(inputs)       
        loss = criterion(outputs, train_labels)  
        iter_loss += loss.item()     
        loss.backward()                
        optimizer.step()       
        _, train_predicted = torch.max(outputs, 1)
        correct += (train_predicted == train_labels).sum()
        #print(t_predicted, t_labels)
    train_loss.append(iter_loss/len(train_dataset_1))
    train_accuracy.append((100 * correct / len(train_dataset_1)))
    print ('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}'.format(epoch+1, num_epochs, train_loss[-1], train_accuracy[-1]))
    

Epoch 1/20, Training Loss: 0.009, Training Accuracy: 29.281
Epoch 2/20, Training Loss: 0.007, Training Accuracy: 49.513
Epoch 3/20, Training Loss: 0.006, Training Accuracy: 59.221
Epoch 4/20, Training Loss: 0.005, Training Accuracy: 65.284
Epoch 5/20, Training Loss: 0.004, Training Accuracy: 69.400
Epoch 6/20, Training Loss: 0.004, Training Accuracy: 72.290
Epoch 7/20, Training Loss: 0.003, Training Accuracy: 74.929
Epoch 8/20, Training Loss: 0.003, Training Accuracy: 76.500
Epoch 9/20, Training Loss: 0.003, Training Accuracy: 78.794
Epoch 10/20, Training Loss: 0.003, Training Accuracy: 80.647
Epoch 11/20, Training Loss: 0.003, Training Accuracy: 80.993
Epoch 12/20, Training Loss: 0.002, Training Accuracy: 82.878
Epoch 13/20, Training Loss: 0.002, Training Accuracy: 84.166
Epoch 14/20, Training Loss: 0.002, Training Accuracy: 83.915
Epoch 15/20, Training Loss: 0.002, Training Accuracy: 84.606
Epoch 16/20, Training Loss: 0.002, Training Accuracy: 86.491
Epoch 17/20, Training Loss: 0.002

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(train_labels.cpu(), train_predicted.cpu()))

              precision    recall  f1-score   support

           0       0.80      0.92      0.86        26
           1       0.94      0.91      0.93        34
           2       0.89      0.83      0.86        30
           3       0.94      0.94      0.94        31
           4       0.93      0.85      0.89        33
           5       0.90      0.97      0.93        29

    accuracy                           0.90       183
   macro avg       0.90      0.90      0.90       183
weighted avg       0.90      0.90      0.90       183



In [ ]:
cnn_model.eval()

Test_accuracy=0
for images, labels in test_loader_1:
    
      # Convert the images and labels to Pytorch tensor 
      test_images = images.to(device)
      test_labels = labels.to(device)

      # Passing images to the model, which return the probabilites as outputs
      test_output = cnn_model(test_images) 

      # Picking the class/label with maximum probability                                                                 
      _, test_predicted = torch.max(test_output, 1)
      Test_accuracy += (test_predicted == test_labels).sum().item()
      #print(predicted, labels)
      #print(Test_accuracy)
#print(classification_report(labels, predicted))
# Calculating the accuracy
Accuracy = 100 * (Test_accuracy/ (len(test_dataset_1)))
print("Accuracy of Test Data is", Accuracy) 


Accuracy of Test Data is 72.11055276381909


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels.cpu(), test_predicted.cpu()))

              precision    recall  f1-score   support

           0       0.82      0.66      0.73        41
           1       0.82      0.68      0.74        34
           2       0.62      0.74      0.68        31
           3       0.68      0.68      0.68        31
           4       0.77      0.77      0.77        30
           5       0.57      0.72      0.64        29

    accuracy                           0.70       196
   macro avg       0.71      0.71      0.70       196
weighted avg       0.72      0.70      0.71       196



In [ ]:
#!unzip '/content/team_data_0.zip'

Archive:  /content/team_data_0.zip
  inflating: team_data/0_b16h1g18_1627628229.wav  
  inflating: team_data/0_b16h1g18_1627629172.wav  
  inflating: team_data/1_b16h1g18_1627628237.wav  
  inflating: team_data/1_b16h1g18_1627629179.wav  
replace team_data/2_b16h1g18_1627628245.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: team_data/2_b16h1g18_1627628245.wav  
  inflating: team_data/2_b16h1g18_1627629188.wav  
  inflating: team_data/3_b16h1g18_1627628252.wav  
  inflating: team_data/3_b16h1g18_1627629195.wav  
  inflating: team_data/4_b16h1g18_1627628260.wav  
  inflating: team_data/4_b16h1g18_1627629204.wav  
  inflating: team_data/5_b16h1g18_1627628267.wav  
replace team_data/5_b16h1g18_1627629210.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: team_data/5_b16h1g18_1627629210.wav  


In [ ]:
for file in glob.glob("/content/team_data/*.wav", recursive=True):
        
    file_name=os.path.basename(file)
    team_labels = (int((file_name.split("/")[-1]).split("_")[0]))
    input_audio_path = '/content/team_data/'+file_name
    f1 = extract_feature(input_audio_path, mfcc=True, chroma=True, mel=True)

    f1 = f1.flatten()[:, np.newaxis]
    f1 = scaler.fit_transform(f1)
    print(f1.shape)
    f1 = Variable(torch.from_numpy(f1)).float()
    image_tensor = f1.unsqueeze(0)
    print(image_tensor.shape)
    m1 = cnn_model.cpu()
    m1.eval()
    output1 = m1(image_tensor)
    print(output1)
    #Uncomment the below lines
    predicted_label = torch.max(output1,1)
    print("PREDICTION",team_labels, predicted_label.indices)
    print('/n')

(180, 1)
torch.Size([1, 180, 1])
tensor([[-2.8790,  2.9924,  1.4342, -3.5858,  0.9565, -1.2948]],
       grad_fn=<AddmmBackward>)
PREDICTION 5 tensor([1])
/n
(180, 1)
torch.Size([1, 180, 1])
tensor([[-2.2339,  1.7659,  1.8079, -2.2965,  0.1804, -1.1701]],
       grad_fn=<AddmmBackward>)
PREDICTION 4 tensor([2])
/n
(180, 1)
torch.Size([1, 180, 1])
tensor([[-2.5501,  2.3199,  1.6991, -2.8624,  0.4215, -1.2640]],
       grad_fn=<AddmmBackward>)
PREDICTION 0 tensor([1])
/n
(180, 1)
torch.Size([1, 180, 1])
tensor([[-2.2669,  1.3992,  2.0741, -1.8879,  0.3798, -1.2981]],
       grad_fn=<AddmmBackward>)
PREDICTION 4 tensor([2])
/n
(180, 1)
torch.Size([1, 180, 1])
tensor([[-2.7500,  2.2527,  1.5466, -2.8903,  0.9464, -0.9490]],
       grad_fn=<AddmmBackward>)
PREDICTION 1 tensor([1])
/n
(180, 1)
torch.Size([1, 180, 1])
tensor([[-2.6881,  2.4507,  1.3006, -3.1409,  0.9223, -0.8665]],
       grad_fn=<AddmmBackward>)
PREDICTION 1 tensor([1])
/n
(180, 1)
torch.Size([1, 180, 1])
tensor([[-2.8514,  

In [ ]:
#state = {'net_dict': cnn_model.state_dict()}
#torch.save(state, 'cnn_model_01.t7')

In [ ]:
#c = z[3500]
#c

'1_G4_96.wav'

In [ ]:
import IPython.display as ipd
ipd.Audio('/content/1_userinput_1.wav') 

# Approach 2

In [ ]:
def get_features(filepath, sr=8000, n_mfcc=30, n_mels=128, frames = 15):
    y, sr = librosa.load(filepath, sr=sr)
    D = np.abs(librosa.stft(y))**2
    S = librosa.feature.melspectrogram(S=D)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    log_S = librosa.power_to_db(S,ref=np.max)
    features = librosa.feature.mfcc(S=log_S, n_mfcc=n_mfcc)
    if features.shape[1] < frames :
        features = np.hstack((features, np.zeros((n_mfcc, frames - features.shape[1]))))
    elif features.shape[1] > frames:
        features = features[:, :frames]
        
    # Find 1st order delta_mfcc
    delta1_mfcc = librosa.feature.delta(features, order=1)

    # Find 2nd order delta_mfcc
    delta2_mfcc = librosa.feature.delta(features, order=2)
    features = np.hstack((delta1_mfcc.flatten(), delta2_mfcc.flatten()))
    features = features.flatten()[np.newaxis, :]
    features = Variable(torch.from_numpy(features)).float()
       
    return features.data.numpy().flatten()

audio_labels = [0, 1, 2, 3, 4, 5]
def load_data_2():
    
    x,y=[],[]

    for file in glob.glob("/content/studio_data/*.wav", recursive=True):
        
        file_name = os.path.basename(file)

        labels = (int((file_name.split("/")[-1]).split("_")[0]))
       
        if labels in audio_labels:
            feature = get_features(file)
            #mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            #feature = np.hstack((feature, mfccs))

            x.append(feature)
            y.append(labels)
            
    return x, y

x2, y2 = load_data_2()

In [ ]:
features2 = np.array(x2)
labels2 = np.array(y2)

In [ ]:
features2.shape, labels2.shape

((3979, 900), (3979,))

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
labels2 = lb.fit_transform(labels2)

from sklearn.preprocessing import StandardScaler
scaler2 = StandardScaler()
features2 = scaler2.fit_transform(features2)

from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2, labels2, test_size = 0.2, random_state=42, shuffle=False)

In [ ]:
# RESHAPE DATA TO INCLUDE 3D TENSOR 
X_train2_1 = X_train2[:,:,np.newaxis]
X_test2_2 = X_test2[:,:,np.newaxis]

X_train2.shape, X_test2.shape, X_train2_1.shape, X_test2_2.shape

((3183, 900), (796, 900), (3183, 900, 1), (796, 900, 1))

In [ ]:
batch_size =500

# The TensorDataset takes an arbitrary number of input tensors.
train_dataset2_1 = torch.utils.data.TensorDataset(torch.FloatTensor(X_train2_1),torch.LongTensor(y_train2))
test_dataset2_1 = torch.utils.data.TensorDataset(torch.FloatTensor(X_test2_2),torch.LongTensor(y_test2))

# Loading the train dataset
train_loader2_1 = torch.utils.data.DataLoader(train_dataset2_1, batch_size=batch_size, shuffle = True)
test_loader2_1 = torch.utils.data.DataLoader(test_dataset2_1, batch_size=batch_size, shuffle = True)

In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=900, out_channels=400, kernel_size=1 )
        self.bn1 = nn.BatchNorm1d(400)
        self.pool1 = nn.MaxPool1d(1)
        self.conv2 = nn.Conv1d(400, 200, 1)
        self.bn2 = nn.BatchNorm1d(200)
        self.pool2 = nn.MaxPool1d(1)
        self.conv3 = nn.Conv1d(200, 128, 1)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(1)
        #self.conv4 = nn.Conv1d(256, 128, 1)
        #self.bn4 = nn.BatchNorm1d(128)
        #self.pool4 = nn.MaxPool1d(1)
        #self.conv5 = nn.Conv1d(256, 128, 1)
        #self.bn5 = nn.BatchNorm1d(128)
        #self.pool5 = nn.MaxPool1d(1)
        self.fc1 = nn.Linear(128, 32)
        #self.fc2 = nn.Linear(128, 64)
        #self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 6)
        self.dropout = nn.Dropout(p=0.25)

    def forward(self, x):
   
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool1(x)
        
        x = self.dropout(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool2(x)
       
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool3(x)
        
        #x = self.conv4(x)
        #x = self.bn4(x)
        #x = F.relu(x)
        #x = self.pool4(x)
        #x = self.dropout(x)

        #x = self.conv5(x)
        #x = self.bn5(x)
        #x = self.pool5(x)
        
        #x = self.dropout(x)
       
        x = x.view(-1, 128)
        x = self.fc1(x)
        #x = self.fc2(x)
        #x = self.fc3(x)
        x = self.fc4(x)
        x = F.log_softmax(x, dim=1)
        return x      
      

In [ ]:
model2 = Model2()
#model = M5()
print(model2)
cnn_model= cnn_model.to(device)
# YOUR CODE HERE to convert your model to runtime cuda

criterion2 = nn.CrossEntropyLoss()# YOUR CODE HERE : Explore and declare loss function

optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.017)

Model2(
  (conv1): Conv1d(900, 400, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(400, 200, kernel_size=(1,), stride=(1,))
  (bn2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(200, 128, kernel_size=(1,), stride=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=6, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [ ]:
num_epochs = 20
model2.train()

#Define the lists to store the results of loss and accuracy
train_loss2 = []
train_accuracy2, test_accuracy2 = [], []

for epoch in range(num_epochs): 
    correct = 0
    iter_loss = 0.0
    
    for (inputs, labels) in train_loader2_1:

        inputs = inputs.to(device)
        t_labels2 = labels.to(device)
        
        optimizer2.zero_grad() 
        outputs2 = model2(inputs)       
        loss = criterion2(outputs2, t_labels2)  
        iter_loss += loss.item()     
        loss.backward()                
        optimizer2.step()       
        _, t_predicted2 = torch.max(outputs2, 1)
        correct += (t_predicted2 == t_labels2).sum()
        #print(t_predicted, t_labels)
    train_loss2.append(iter_loss/len(train_dataset2_1))
    train_accuracy2.append((100 * correct / len(train_dataset2_1)))
    print ('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}'.format(epoch+1, num_epochs, train_loss2[-1], train_accuracy2[-1]))
    

Epoch 1/20, Training Loss: 0.004, Training Accuracy: 28.998
Epoch 2/20, Training Loss: 0.002, Training Accuracy: 58.278
Epoch 3/20, Training Loss: 0.002, Training Accuracy: 74.615
Epoch 4/20, Training Loss: 0.001, Training Accuracy: 79.548
Epoch 5/20, Training Loss: 0.001, Training Accuracy: 84.417
Epoch 6/20, Training Loss: 0.001, Training Accuracy: 88.533
Epoch 7/20, Training Loss: 0.001, Training Accuracy: 90.732
Epoch 8/20, Training Loss: 0.000, Training Accuracy: 92.931
Epoch 9/20, Training Loss: 0.000, Training Accuracy: 94.094
Epoch 10/20, Training Loss: 0.000, Training Accuracy: 93.811
Epoch 11/20, Training Loss: 0.000, Training Accuracy: 94.094
Epoch 12/20, Training Loss: 0.000, Training Accuracy: 95.130
Epoch 13/20, Training Loss: 0.000, Training Accuracy: 95.947
Epoch 14/20, Training Loss: 0.000, Training Accuracy: 96.418
Epoch 15/20, Training Loss: 0.000, Training Accuracy: 96.953
Epoch 16/20, Training Loss: 0.000, Training Accuracy: 96.890
Epoch 17/20, Training Loss: 0.000

In [ ]:
model2.eval()

Test_accuracy2=0
for images, labels in test_loader2_1:
    
      # Convert the images and labels to Pytorch tensor 
      test_images = images.to(device)
      test_labels2 = labels.to(device)

      # Passing images to the model, which return the probabilites as outputs
      test_output2 = model2(test_images) 

      # Picking the class/label with maximum probability                                                                 
      _, test_predicted2 = torch.max(test_output2, 1)
      Test_accuracy2 += (test_predicted2 == test_labels2).sum().item()
      #print(predicted, labels)
      #print(Test_accuracy)
#print(classification_report(labels, predicted))
# Calculating the accuracy
Accuracy = 100 * (Test_accuracy2/ (len(test_dataset2_1)))
print("Accuracy of Test Data is", Accuracy) 

Accuracy of Test Data is 81.28140703517587


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels2.cpu(), test_predicted2.cpu()))

              precision    recall  f1-score   support

           0       0.70      0.78      0.74        45
           1       0.88      0.83      0.85        60
           2       0.80      0.66      0.73        50
           3       0.87      0.80      0.84        41
           4       0.81      0.88      0.84        58
           5       0.72      0.81      0.76        42

    accuracy                           0.80       296
   macro avg       0.80      0.79      0.79       296
weighted avg       0.80      0.80      0.80       296



In [ ]:
for file in glob.glob("/content/team_data/*.wav", recursive=True):
        
    file_name=os.path.basename(file)
    team_labels = (int((file_name.split("/")[-1]).split("_")[0]))
    input_audio_path = '/content/team_data/'+file_name
    f2 = get_features(input_audio_path)

    f2 = f2.flatten()[:, np.newaxis]
    f2 = scaler.fit_transform(f2)
    print(f2.shape)
    f2 = Variable(torch.from_numpy(f2)).float()
    image_tensor2 = f2.unsqueeze(0)
    print(image_tensor2.shape)
    m2 = model2.cpu()
    m2.eval()
    output20 = m2(image_tensor2)
    print(output20)
    #Uncomment the below lines
    predicted_label2 = torch.max(output20,1)
    print("PREDICTION",team_labels, predicted_label2.indices)
    print('/n')

(900, 1)
torch.Size([1, 900, 1])
tensor([[-2.6278e+01, -1.4033e+01, -2.5806e+01, -1.7168e+01, -2.2461e+01,
         -8.3446e-07]], grad_fn=<LogSoftmaxBackward>)
PREDICTION 5 tensor([5])
/n
(900, 1)
torch.Size([1, 900, 1])
tensor([[-2.2066e+01, -4.5831e+00, -2.1366e+01, -2.4878e+01, -1.4972e-02,
         -5.3737e+00]], grad_fn=<LogSoftmaxBackward>)
PREDICTION 4 tensor([4])
/n
(900, 1)
torch.Size([1, 900, 1])
tensor([[ -7.1555, -11.7024,  -0.9767, -18.0067,  -0.4737, -17.2812]],
       grad_fn=<LogSoftmaxBackward>)
PREDICTION 0 tensor([4])
/n
(900, 1)
torch.Size([1, 900, 1])
tensor([[-1.5919e+01, -9.1074e+00, -1.1489e+01, -1.9872e+01, -1.2278e-04,
         -1.3393e+01]], grad_fn=<LogSoftmaxBackward>)
PREDICTION 4 tensor([4])
/n
(900, 1)
torch.Size([1, 900, 1])
tensor([[-15.6239,  -0.0594, -13.7913, -15.5501, -12.6664,  -2.8536]],
       grad_fn=<LogSoftmaxBackward>)
PREDICTION 1 tensor([1])
/n
(900, 1)
torch.Size([1, 900, 1])
tensor([[-17.7269,  -0.5115, -17.9959, -15.3617, -17.4260,  -0

In [ ]:
state = {'net_dict': model2.state_dict()}
torch.save(state, 'model2.t7')